In [200]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [207]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

In [242]:
# Initialize variables
req_header = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive' }

df = pd.read_csv('consultant_list_consolidated.csv')

code = []
gmc_code = []
consultant_name = []
specialty = []
specialise_in = []
specialise_in2 = []
badges = []
practise_at_n = []

In [241]:
test_url = 'https://www.finder.bupa.co.uk/Consultant/View/80'
driver.get(test_url)
wait = WebDriverWait(driver, 10)
element = wait.until(EC.presence_of_element_located((By.ID, "map-info-holder")))
page = BeautifulSoup(driver.page_source, 'html.parser')
sp = page.find(id = 'map-info-holder')
sp = sp.find('h4')
sp = sp.get_text().split('   ', 1)[0].strip()
print(' '.join(sp.split()))

Practises at 2 hospitals/clinics


In [243]:
# Full scraping loop
for i, j in enumerate(df['Code']):
    test_url = 'https://www.finder.bupa.co.uk/Consultant/View/' + str(j)
    test_request = requests.get(url=test_url, headers=req_header)
    if test_request.status_code == 200:
        page = BeautifulSoup(test_request.text, 'html.parser')

        # GMC Code
        # gmc = page.find(id = 'pnum')
        # gmc_code.append(gmc.get_text())

        #Consultant Name
        cn = page.find(class_ = 'cnwrpt')
        consultant_name.append(cn.get_text())

        # GMC Code & Specialty
        sp = page.find('h3')
        sp = sp.get_text().split(' ',1)[1]
        sp = sp.strip().split('  ', 1)
        specialty.append(sp[0].strip())
        gmc_code.append(sp[1].strip())

        #Specialises in & BUPA Specialist
        sp_in = page.findAll('ul', class_ = 'list third')
        if sp_in:
            specialise_in.append(sp_in[0].get_text().strip())
            specialise_in2.append(sp_in[1].get_text().strip())
        else:
            specialise_in.append(np.nan)
            specialise_in2.append(np.nan)

        #Badges
        bdg = page.find(class_ = 'has-is-box')
        bdg = bdg.findAll('li')
        badges_text = ''
        for k in bdg:
            k = k.get_text()
            k = ' '.join(k.split())
            if badges_text:
                badges_text = '\n'.join([badges_text, k])
            else:
                badges_text = badges_text + k
        badges.append(badges_text)

        #Practises at N facilities
        driver.get(test_url)
        wait = WebDriverWait(driver, 2)
        element = wait.until(EC.presence_of_element_located((By.ID, "map-info-holder")))
        page2 = BeautifulSoup(driver.page_source, 'html.parser')
        prac_at_n = page2.find(id = 'map-info-holder')
        prac_at_n = prac_at_n.find('h4')
        prac_at_n = prac_at_n.get_text().split('   ', 1)[0].strip()
        practise_at_n.append(' '.join(prac_at_n.split()))

    if i == 2:
        break


In [ ]:
# Store in dataframe
df_new = pd.DataFrame()
df_new['Page Code'] = df['Code'][:i+1]
df_new['GMC Code'] = gmc_code
df_new['Name'] = consultant_name
df_new[['Title', 'First Name', 'Last Name']] = df_new['Name'].str.split(' ', 2, expand=True)
df_new['Specialty'] = specialty
df_new['Specialise In'] = specialise_in
df_new['BUPA Specialist'] = specialise_in2
df_new['Badges'] = badges
df_new['Practises at N facilities'] = practise_at_n
df_new.head()

,Page Code,GMC Code,Name,Title,First Name,Last Name,Specialty,Specialise In,BUPA Specialist,Badges
0,80,00003883,Mr Ian J Bayley,Mr,Ian,J Bayley,Trauma & orthopaedic surgery,NaN,NaN,Not Fee assured\nAwaiting verification
1,385,00041676,Dr Maurice Atkins,Dr,Maurice,Atkins,General psychiatry,NaN,NaN,Fee assured\nAwaiting verification
2,628,00070243,Dr Hedley Berry,Dr,Hedley,Berry,Rheumatology,NaN,NaN,Fee assured\nVerified account


In [ ]:
# df_new.to_csv('consultant_directory.csv') 